# Python Decorators

http://tooblippe.github.io/insightstack-blog/2014/05/05/decorators/

This document captures the essential elements of decorators, based on the work of others.  The text here is brief, please consult the references for more details.

## Decorators, why?

- add functionality to existing functions and classes.
- do timing on functions.
- perform pre- and post-formatting on data.
- diagnostics and testing.
- perform monkey patching.


## Decorator concepts

In http://simeonfranklin.com/blog/2012/jul/1/python-decorators-in-12-steps/ Franklin provides the ground work for building up to the punchline

1\. It all starts with Python functions that has a name, an optional list of parameters (positional and named/default) and can return a value.

2\. Variables defined inside a function has scope only inside the function.

3\. Variable names are resolved (for access) with the [LEGB](http://nbviewer.ipython.org/github/rasbt/python_reference/blob/master/tutorials/scope_resolution_legb_rule.ipynb) (local, enclosing, global, built-in) rule. The first name found is used.  Inner scopes have read access to outer scopes, but not write/change access.

4\. Variables have lifetime only in the scope that exists at any moment.

5\. Variables passed as function parameters become local variables in the function.

6\. Python allows nested functions, with the normal scoping rules. In this code `inner` is a local variable in the function `outer`, it just happens to be a function:

In [ ]:
def outer():
    x = 1
    def inner():
        print(x)
    inner()
    
outer()    

7\. Functions are objects that/(whose names) can be passed as variables to functions and can be returned from functions. 

In [ ]:
def add(x, y):
    return x + y
def apply(func, x, y):
    return func(x, y)
apply(add, 2, 1)

In this example, `inner` is redefined every time when `outer()` is executed. The function `outer` returns the function name `inner` which can be called with `()`.

In [ ]:
def outer():
    def inner():
        print('Inside inner')
    return inner

print(outer)
print(outer())
outer()
outer()()


8\. Python supports function closure, which means that non-global functions remember what their enclosing namespaces looked like at definition time, even if the enclosing environment (function in this case) goes out of scope.   In this example the variable `x` goes out of scope when `outer()` returns `inner` to the variable `foo`, but `inner` still remembers the value when `foo()` is executed.

In [ ]:
def outer():
    x = 1
    def inner():
        print(x)
    return inner
foo = outer()
# x is out of scope at this point
print(foo.__closure__) #foo.func_closure in Python2
foo()

Here is an example of how closure can build custom functions:

In [ ]:
def outer(x):
    def inner():
        print(x)
    return inner
print1 = outer(1)
print2 = outer(2)
print1()
print2()


9\. A decorator is a function that takes a function name as an argument and returns a replacement function, which can be called elsewhere. Here is the principle at work for a function with no parameters:

In [ ]:
def outer(some_func):
    def inner():
        print('before some_func - do whatever you want before')
        ret = some_func()
        print('after some_func - do whatever you want after')
        return ret + 1
    return inner
def foo():
    return 1
decorated = outer(foo)
decorated()

If we want to keep the `foo` function name, just reassign it to the decorated function, then calling `foo` will call the decorated function.

Caveat IPython cell memory!  Run the following cell a few times....

In [ ]:
foo = outer(foo) # now becomes outer(foo), same as decorated above
print(foo)
foo() 

10\. The @ symbol applies a decorator to a function, as in 

    @decoratorname
    def function():
        pass

11\. `*args` and `**kwargs` provides the means to pass along any number of function parameters.  This allows one to wrap/decorate any function, irrespective of its signature.  
By using `*args` (list of function arguments) and `**kwargs` (dictionary of function keyword arguments) the decorator can be applied to functions and class methods alike, with any number of arguments.  The `*args` list  and `**kwargs` dictionary are just unrolled into the new `func` call.

12\. Simple example: printing function parameters before calling a function.

In [ ]:
def logger(func):
    def inner(*args, **kwargs): 
        print('Arguments were: {}, {}'.format(args, kwargs))
        rtnVal = func(*args, **kwargs)
        print('Return value is: {}'.format(rtnVal))
        return rtnVal
    return inner

In [ ]:
@logger
def foo1(x,y=1):
    return x * y
@logger
def foo2():
    return 2

foo1(5,4)
foo2()


13\. Here is an example by [Farhad](http://thecodeship.com/patterns/guide-to-python-function-decorators/) of the decorator applied to a class method:

In [ ]:
def p_decorate(func):
   def inner(*args, **kwargs):
       return "<p>{0}</p>".format(func(*args, **kwargs))
   return inner

class Person(object):
    def __init__(self):
        self.name = "John"
        self.family = "Doe"

    @p_decorate
    def get_fullname(self):
        return self.name+" "+self.family

my_person = Person()

print(my_person.get_fullname)


### Passing arguments to decorators

Decorators can be passed arguments by using an *additional* level of function nesting around the decorators seen thus far.

This [Farhad](http://thecodeship.com/patterns/guide-to-python-function-decorators/) example passes arguments to the decorator function:


In [ ]:
def tags(tag_name):
    def tags_decorator(func):
        def inner(name):
            return '<{0}>{1}</{0}>'.format(tag_name, func(name))
        return inner
    return tags_decorator

@tags('p')
@tags('strong')
def get_text(name):
    return 'Hello '+name

print(get_text('John'))


You can also pass conditionals to decorators to change behaviour.

In [ ]:
def skipIf(conditional, message):
    def dec(func):
        def inner(*args, **kwargs):
            if conditional:
                print(message)
            else:
                return func(*args, **kwargs)
        return inner
    return dec

logical = [True, False]
for log in logical:
    print(log)
    @skipIf(log, 'skipped function call')
    def myFunc():
        print('in myFunc')
    myFunc()

## Decorators and function attributes

The decorating wrapper as shown above overwrites the function's attributes, so the wrapped function's attributes are lost.

As of Python 2.5, the functools module contains `functools.wraps`, a decorator for updating the attributes of the wrapping function to those of the original function. This is as simple as decorating the inner function with `@wraps(func)`. [Farhad's](http://thecodeship.com/patterns/guide-to-python-function-decorators/) example:

In [ ]:
from functools import wraps

def tags(tag_name):
    def tags_decorator(func):
        @wraps(func)
        def inner(name):
            return "<{0}>{1}</{0}>".format(tag_name, func(name))
        return inner
    return tags_decorator

@tags("p")
def get_text(name):
    """returns some text"""
    return "Hello "+name

print(get_text.__name__) # get_text
print(get_text.__doc__) # returns some text
print(get_text.__module__) # __main__


Graham Dumpleton created a library called [`wrapt`](https://pypi.python.org/pypi/wrapt), going much beyond `wraps`. Dumpleton writes: "To  provide a transparent object proxy for Python, which can be used as the basis for the construction of function wrappers and decorator functions. 
It therefore goes way beyond existing mechanisms such as functools.wraps() to ensure that decorators preserve introspectability, signatures, type checking abilities etc. The decorators that can be constructed using this module will work in far more scenarios than typical decorators and provide more predictable and consistent behaviour." See the use of this module below.

## Decorator classes

The decorator wrapper must be a callable, such as a function (shown above) or a class with a `__call__()` member function. Hence classes can also be used as decorators.  Class decorators are slightly more involved than function decorators, but implements the same functionality. See [Eckel's blogs](https://www.artima.com/weblogs/viewpost.jsp?thread=240845) 

When using the class as a decorator, the class is initialised and remembers the original function, whereas function decorators reassign the original function name to the newly wrapped function name.

Use `functools.update_wrapper()` to make available the wrapped functions attributes to the decorator wrapping class.


In [ ]:
from functools import update_wrapper

class myDecorator(object):

    def __init__(self, f):
        print('inside myDecorator.__init__()')
        self.f = f
        update_wrapper(self, f)

    def __call__(self):
        print("Entering", self.f.__name__)
        self.f()
        print("Exited", self.f.__name__)

@myDecorator
def aFunction():
    """aFunction does aThing"""
    print('inside aFunction()')

print('Finished decorating aFunction()\n')

aFunction()

print('\nWrapped function name is {}'.format(aFunction.__name__))
print('Wrapped docstring is "{}"'.format(aFunction.__doc__))


Class decorators can also have parameters. This time the decorator class implementation is a lot more complex than the decorator function implementation.

In [ ]:
class decoratorWithArguments(object):

    def __init__(self, arg1, arg2, arg3):
        """
        If there are decorator arguments, the function
        to be decorated is not passed to the constructor!
        """
        print('Inside __init__()')
        self.arg1 = arg1
        self.arg2 = arg2
        self.arg3 = arg3

    def __call__(self, f):
        """
        If there are decorator arguments, __call__() is only called
        once, as part of the decoration process! You can only give
        it a single argument, which is the function object.
        """
        print('Inside __call__()')
        def wrapped_f(*args):
            print('Inside wrapped_f()')
            print('Decorator arguments: {} {} {} '.format(self.arg1, self.arg2, self.arg3))
            f(*args)
            print('After f(*args)')
        return wrapped_f

@decoratorWithArguments("hello", "world", 42)
def sayHello(a1, a2, a3, a4):
    print('sayHello arguments: {} {} {} {}'.format(a1, a2, a3, a4))

print('After decoration')

print('Preparing to call sayHello()')
sayHello("say", "hello", "argument", "list")
print('after first sayHello() call')
sayHello("a", "different", "set of", "arguments")
print('after second sayHello() call')

This seems overly complex and confusing, but I guess it might be required for some sophisticated applications. In [Eckel's](https://www.artima.com/weblogs/viewpost.jsp?thread=240845) own words:

"Now the process of decoration calls the constructor and then immediately invokes `__call__()`, which can only take a single argument (the function object) and must return the decorated function object that replaces the original. Notice that `__call__()` is now only invoked once, during decoration, and after that the decorated function that you return from `__call__()` is used for the actual calls."

"Although this behavior makes sense -- the constructor is now used to capture the decorator arguments, but the object `__call__()` can no longer be used as the decorated function call, so you must instead use `__call__()` to perform the decoration -- it is nonetheless surprising the first time you see it because it's acting so much differently than the no-argument case, and you must code the decorator very differently from the no-argument case."

## Decorating classes

All the examples thus far decorated/wrapped functions.  Can classes be decorated?  Based on this blog by [da Palma](http://andrefsp.wordpress.com/2012/08/23/writing-a-class-decorator-in-python/), it seems that there are two options: (1) decorating a class by decorating each of its member functions and (2) where you write a class decorator and select which methods to decorate passing their names as the decorator arguments.
I repeat his code below exactly as blogged.

In [ ]:
def method_decorator(fn):
    "Example of a method decorator"
    def decorator(*args, **kwargs):
        print("\tInside the decorator")
        return fn(*args, **kwargs)

    return decorator

class MyFirstClass(object):
    """
    This class has all its methods decorated
    """
    @method_decorator
    def first_method(self, *args, **kwargs):
        print("\t\tthis is a the MyFirstClass.first_method")

    @method_decorator
    def second_method(self, *args, **kwargs):
        print("\t\tthis is the MyFirstClass.second_method")

if __name__ == "__main__":
    print("::: With decorated methods :::")
    x = MyFirstClass()
    x.first_method()
    x.second_method()

In [ ]:
def method_decorator(fn):
    "Example of a method decorator"
    def decorator(*args, **kwargs):
        print("\tInside the decorator")
        return fn(*args, **kwargs)

    return decorator


def class_decorator(*method_names):
    def class_rebuilder(cls):
        "The class decorator example"
        class NewClass(cls):
            "This is the overwritten class"
            def __getattribute__(self, attr_name):
                obj = super(NewClass, self).__getattribute__(attr_name)
                if hasattr(obj, '__call__') and attr_name in method_names:
                    return method_decorator(obj)
                return obj

        return NewClass
    return class_rebuilder


@class_decorator('first_method', 'second_method')
class MySecondClass(object):
    """
    This class is decorated
    """
    def first_method(self, *args, **kwargs):
        print("\t\tthis is a the MySecondClass.first_method")

    def second_method(self, *args, **kwargs):
        print("\t\tthis is the MySecondClass.second_method")

if __name__ == "__main__":
    print("::: With a decorated class :::")
    z = MySecondClass()
    z.first_method()
    z.second_method()

## Dumpleton's `warpt` module

I discoved [Dumpleton's work](http://wrapt.readthedocs.org/en/latest/) in his 
[Pycon 2014 video](https://www.youtube.com/watch?v=7jGtDGxgwEY), after all of the above was written. The writeup above is still valid but it is recommended that you use the [wrapt module](https://pypi.python.org/pypi/wrapt) for production work on decorators, wrappers and monkey patching. This module provides the required  robustness to decorators to behave as expected (e.g., as functions) under all conditions.  

Dumpleton shows in his talk how the techniques described above have weaknesses when it comes to accessing the wrapped function's attributes and descriptors. During the talk, Dumpleton smiled and made the remark "totally losing you at this point, I guess" and "I will let you digest that one...". This is indeed the case when he goes into the very deep details of fixing the weaknesses of the simple decorators.  Fortunately you don't have to understand the inner details to use the [wrapt module](https://pypi.python.org/pypi/wrapt).

Dumpleton's estimate is that 90% of all decorator requirements can be met by simple function closure wrappers (as described above) - but you lose significant introspection functionality. He recommends that you use [wrapt](http://wrapt.readthedocs.org/en/latest/) for developing libraries that will be used by other people.

The `wrapt` module results in more compact code and full introspection functionality; no `inner` function coding is required.

In [ ]:
import wrapt

@wrapt.decorator
def pass_through(wrapped, instance, args, kwargs):
    return wrapped(*args, **kwargs)

@pass_through
def function():
    """Function docstring"""
    print('Using wrapt with a decorator function and no decorator arguments')

function()
print('Function name'.format(function.__name__))
print('Function doctring'.format(function.__doc__))


In [ ]:
import wrapt

class with_arguments(object):

    def __init__(self, myarg1, myarg2):
        self.myarg1 = myarg1
        self.myarg2 = myarg2

    @wrapt.decorator
    def __call__(self, wrapped, instance, args, kwargs):
        return wrapped(*args, **kwargs)

@with_arguments(1, 2)
def function():
    print('Using wrapt with a decorator class and decorator arguments')

function()

`wrapt` decorators can be enabled and disabled by setting a flag.  If enabled, the wrapped function is returned, if disabled, the original function is returned.

In [ ]:
for ENABLED in [True, False]:
    print('\nDecorator enabled is {}'.format(ENABLED))

    @wrapt.decorator(enabled=ENABLED)
    def pass_through(wrapped, instance, args, kwargs):
        print('In decorator, before calling function')
        wraptfun = wrapped(*args, **kwargs)
        print('In decorator, after calling function')
        return wraptfun

    @pass_through
    def function():
        print('Executing function')
    print(type(function))
    function()



## Decorator applications

### Adding names to `__all__`

<http://code.activestate.com/recipes/576993-public-decorator-adds-an-item-to-__all__/>
The DRY principle says, "Don't repeat yourself". A Python module typically defines a global variable named "__all__" that holds the names of everything the author wants visible. This means you have to type each function or class name a second time, and you have to maintain the contents of __all__ manually. The `@public` decorator fixes that.    
    

In [ ]:
import sys

def public(f):
    """"Use a decorator to avoid retyping function/class names.

    * Based on an idea by Duncan Booth:
    http://groups.google.com/group/comp.lang.python/msg/11cbb03e09611b8a
    * Improved via a suggestion by Dave Angel:
    http://groups.google.com/group/comp.lang.python/msg/3d400fb22d8a42e1
    """
    all = sys.modules[f.__module__].__dict__.setdefault('__all__', [])
    if f.__name__ not in all:  # Prevent duplicates if run from an IDE.
        all.append(f.__name__)
    return f
public(public)  # Emulate decorating ourself


I've seen two techniques for exporting variables. One is to have a simple assignment somewhere (usually at the beginning or end of the module): `__all__ = [ 'foo', 'bar' ]`

This separates the exportation from the item's definition, so other people do this:
     __all__ = []
     ... 
     def foo(...): 
     ... 
     __all__.append('foo')
     class bar(...):
     ... 
     __all__.append('bar')

This is better, but it still puts the code at the bottom of the item's declaration.

My solution is this:

    import public from my_utilites

    @public def foo(...): ...

    @public class bar(...): ...


### Context managers

This example is a very simple example of applying the `@contextmanager` decorator. The program flow is really simple: you (1) create some context, (2) yield when the context is used and (3) finalise your work when closing the context.  the example below is a simple implementation of the `with open() as f` idiom using a function.

In [ ]:
from contextlib import contextmanager

@contextmanager
def myfile(filename, filemode):
    f = open(filename, filemode)
    try:
        yield f
    finally:
        f.close()
                    
with myfile('test.txt','w') as mf:
    mf.write('one line\n')

import os.path    
if os.path.sep == '/':
    !cat test.txt
else:
    !type test.txt


The context manager can also use a class to set up and finalise the context.

In [ ]:
class MyFile:
    def __init__(self, filename, mode):
        self.filename = filename
        self.mode = mode
 
    def __enter__(self):
        self.thefile = open(self.filename, self.mode)
        return self.thefile
 
    def __exit__(self, *unused):
        self.thefile.close()
 
with MyFile('test2.txt','w') as writer:
    writer.write("Hello World from our new Context Manager!")
    
import os.path    
if os.path.sep == '/':
    !cat test2.txt
else:
    !type test2.txt


### Timing with decorators

In [ ]:
import math
import time

def time_dec(func):
  def inner(*args, **kwargs):
      t = time.clock()
      res = func(*args, **kwargs)
      print('fn={} time={}'.format(func.__name__, time.clock()-t))
      return res
  return inner

@time_dec
def myFunction(a):
    return math.atan(a)
    
print(myFunction(0.55))    
    

### Chaining decorators

Chaining decorators is simply wrapping decorators within decorators, within decorators....

In [ ]:
def makebold(fn):
    def inner(*args, **kwargs):
        return "<b>" + fn(*args, **kwargs) + "</b>"
    return inner

def makeitalic(fn):
    def inner(*args, **kwargs):
        return "<i>" + fn(*args, **kwargs) + "</i>"
    return inner

@makebold
@makeitalic
def hello():
    return 'hello world'

print(hello())

### Counting the number of times a function is called

In [ ]:
def count(func):
  def inner(*args, **kwargs):
      counter += 1
      return func(*args, **kwargs)
  counter = 0
  return inner

@count
def myFunction():
    pass
    
myFunction()
myFunction()

print(myFunction.counter)    

### Redirecting stdout to logger

In [ ]:
import logging
import sys

class LogPrinter:
    '''LogPrinter class which serves to emulates a file object and logs
       whatever it gets sent to a Logger object at the INFO level.'''
    def __init__(self):
        '''Grabs the specific logger to use for logprinting.'''
        self.ilogger = logging.getLogger('logprinter')
        il = self.ilogger
        logging.basicConfig()
        il.setLevel(logging.INFO)
        hdlr = logging.FileHandler('logprinter.log')
        formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s')
        hdlr.setFormatter(formatter)
        self.ilogger.addHandler(hdlr) 

    def write(self, text):
        '''Logs written output to a specific logger'''
        self.ilogger.info(text)

def logprintinfo(func):
    '''Wraps a method so that any calls made to print get logged instead'''
    def pwrapper(*arg, **kwargs):
        stdobak = sys.stdout
        lpinstance = LogPrinter()
        sys.stdout = lpinstance
        try:
            return func(*arg, **kwargs)
        finally:
            sys.stdout = stdobak
    return pwrapper

@logprintinfo
def somePrintingFunc():
    print('Hello output')
    
somePrintingFunc()

import os.path    
if os.path.sep == '/':
    !cat logprinter.log
else:
    !type logprinter.log


### A decorator-based build system

[Bruce Eckel](https://www.artima.com/weblogs/viewpost.jsp?thread=241209) built a build system using python decorators. His motivation is that build systems really need a language and that `make` and `ant` do not supply sufficient capability in this area.  In my own view [CMake](http://www.cmake.org/) is the better way to go here - is has a horrible language, but it works.

### Repository of decorators

<https://wiki.python.org/moin/PythonDecoratorLibrary> has a large number of decorators for many different applications. The following is a list of some of these.  The code won't work here, because some of the decorators are quite long and must be included in your code.

Use the `accepts` decorator for type checking:  

    @accepts(uint, utf8string)
    def myMethod(ID, name):
        print(ID, name)

    myMethod(4, 'string')

Implement singletons:  

    @singleton
    class Highlander:
        x = 100

    Highlander() is Highlander() is Highlander #=> True
    id(Highlander()) == id(Highlander) #=> True
    Highlander().x == Highlander.x == 100 #=> True
    Highlander.x = 50
    Highlander().x == Highlander.x == 50 #=> True

Memoizing a class. Caches a function's return value each time it is called. If called later with the same arguments, the cached value is returned (not reevaluated)

    @memoized
    def fibonacci(n):
       "Return the nth fibonacci number."
       if n in (0, 1):
          return n
       return fibonacci(n-1) + fibonacci(n-2)

    print fibonacci(12)


Smart deprecation warnings (with valid filenames, line numbers, etc.)

    @deprecated
    def my_func():
        pass

    my_func()

In [14]:
#minitask 6.1

@deprecated
def some_old_function(x, y):
    return x + y

some_old_function(1,2)

Call to deprecated function: some_old_function


3

In [9]:
import deprecation

__version__ = '1.8'

@deprecation.deprecated(deprecated_in="1.0", removed_in="2.0",
                        current_version=__version__,
                        details="Use the bar function instead")
def foo():
    """Do some stuff"""
    return 1

foo()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecatedWarning: foo is deprecated as of 1.0 and will be removed in 2.0. Use the bar function instead
  if sys.path[0] == '':


1

Easy Dump of Function Arguments

    @dump_args
    def f1(a,b,c):
        print a + b + c

    f1(1, 2, 3)

State machine implementations  
<https://wiki.python.org/moin/PythonDecoratorLibrary#State_Machine_Implementaion>  
<https://wiki.python.org/moin/State%20Machine%20via%20Decorators>

Access control prevents users from getting access to places where they are not authorised to go.

    @LoginCheck
    def display_members_page():
        print 'This is the members page'

## References

Pycon 2014 videos:  
[Myers: Decorators: A Powerful Weapon in your Python Arsenal](https://www.youtube.com/watch?v=9oyr0mocZTg)  
[Dumpleton: Advanced methods for creating decorators](https://www.youtube.com/watch?v=7jGtDGxgwEY)  
[Harrison: Hands-on Intermediate Python](https://www.youtube.com/watch?v=29iDgal40hc)

The definitive work on decorators:  
[Dumpleton: wrapt package](https://pypi.python.org/pypi/wrapt)  
[Dumpleton: wrapt documentation](http://wrapt.readthedocs.org/en/latest/)  

Decorator repository:  
[wiki.python.org](https://wiki.python.org/moin/PythonDecoratorLibrary)  

Blogs:  
[Knupp: Improve Your Python: Decorators Explained](https://www.jeffknupp.com/blog/2013/11/29/improve-your-python-decorators-explained/)  
[Franklin: Understanding Python Decorators in 12 Easy Steps!](http://simeonfranklin.com/blog/2012/jul/1/python-decorators-in-12-steps/)  
[Eckel: Decorators I: Introduction to Python Decorators](https://www.artima.com/weblogs/viewpost.jsp?thread=240808)  
[Eckel: Python Decorators II: Decorator Arguments](https://www.artima.com/weblogs/viewpost.jsp?thread=240845)  
[Eckel: Python Decorators III: A Decorator-Based Build System](https://www.artima.com/weblogs/viewpost.jsp?thread=241209)  
[Farhat: A guide to Python's function decorators](http://thecodeship.com/patterns/guide-to-python-function-decorators/)  
[Yasoob: Python decorators finally demystified](http://freepythontips.wordpress.com/2013/12/05/python-decorators-finally-demystified/)  
[Simionato: PyPI decorator module](https://pypi.python.org/pypi/decorator)  
[da Palma: Writing a class decorator in Python](http://andrefsp.wordpress.com/2012/08/23/writing-a-class-decorator-in-python/)  
[Harrison: Guide to learning Python decorators](http://hairysun.com/books/decorators/)  
[Hellman: contextlib â Context manager utilities](http://pymotw.com/2/contextlib/)  
[Context managers in Python](http://pypix.com/python/context-managers/)  
[Python wiki: PythonDecorators](https://wiki.python.org/moin/PythonDecorators)  
[Chaining decorators](https://stackoverflow.com/questions/739654/how-can-i-make-a-chain-of-function-decorators-in-python)  
[Some common uses](https://stackoverflow.com/questions/489720/what-are-some-common-uses-for-python-decorators)  
[Python decorators finally demystified](http://freepythontips.wordpress.com/2013/12/05/python-decorators-finally-demystified/)  

[Advanced Use of Python Decorators and Metaclasses](http://lgiordani.com/blog/2014/10/14/decorators-and-metaclasses/)


# Performance analysis of Python programs

https://wiki.python.org/moin/PythonSpeed/PerformanceTips#Profiling_Code

**cProfile** and **profile** modules provide deterministic profiling of Python programs

A profile is a set of statistics that describes how often and for how long various parts of the program executed

These statistics can be formatted into reports via the pstats module

**cProfile** is recommended for most users; itâs a C extension with reasonable overhead that makes it suitable for profiling long-running programs

**profile**, a pure Python module whose interface is imitated by cProfile, but which adds significant overhead to profiled programs

The profiler modules are designed to provide an execution profile for a given program, not for benchmarking purposes

For that, there is **timeit** for reasonably accurate results

The most basic starting point in the profile module is **run()**

It takes a string statement as argument, and creates a report of the time spent executing different lines of code while running the statement

In [ ]:
def f(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return f(n-1) + f(n-2)

In [ ]:
import cProfile

def fib(n):
    # from http://en.literateprograms.org/Fibonacci_numbers_(Python)
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)
cProfile.run('print(fib(20))')

In [ ]:
import cProfile

def fib_seq(n):
    seq = [ ]
    if n > 0:
        seq.extend(fib_seq(n-1))
    seq.append(fib(n))
    return seq

cProfile.run('print(fib_seq(20))')

In [ ]:
import cProfile

def fib(n):
    # from http://en.literateprograms.org/Fibonacci_numbers_(Python)
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)

def fib_seq(n):
    seq = [ ]
    if n > 0:
        seq.extend(fib_seq(n-1))
    seq.append(fib(n))
    return seq

cProfile.run('print(fib_seq(20))')

In [ ]:
class m1:
    def __init__(self, function):
        self.function = function
        self.m = {}
    def __call__(self, *args):
        try:
            return self.m[args]
        except KeyError:
            self.m[args] = self.function(*args)
            return self.m[args]


In [ ]:
class memoize:
    def __init__(self, function):
        self.function = function
        self.memoized = {}
    def __call__(self, *args):
        try:
            return self.memoized[args]
        except KeyError:
            self.memoized[args] = self.function(*args)
            return self.memoized[args]


In [ ]:
import collections
import functools

class memoized(object):
   '''Decorator. Caches a function's return value each time it is called.
   If called later with the same arguments, the cached value is returned
   (not reevaluated).
   '''
   def __init__(self, func):
      self.func = func
      self.cache = {}
   def __call__(self, *args):
      if not isinstance(args, collections.Hashable):
         # uncacheable. a list, for instance.
         # better to not cache than blow up.
         return self.func(*args)
      if args in self.cache:
         return self.cache[args]
      else:
         value = self.func(*args)
         self.cache[args] = value
         return value
   def __repr__(self):
      '''Return the function's docstring.'''
      return self.func.__doc__
   def __get__(self, obj, objtype):
      '''Support instance methods.'''
      return functools.partial(self.__call__, obj)

In [ ]:
# note that this decorator ignores **kwargs
def memoize(obj):
    cache = obj.cache = {}

    @functools.wraps(obj)
    def memoizer(*args, **kwargs):
        if args not in cache:
            cache[args] = obj(*args, **kwargs)
        return cache[args]
    return memoizer

In [ ]:
# note that this decorator ignores **kwargs
def memoize(obj):
    cache = obj.cache = {}

    @functools.wraps(obj)
    def memoizer(*args, **kwargs):
        if args not in cache:
            cache[args] = obj(*args, **kwargs)
        return cache[args]
    return memoizer

In [ ]:
def memoize(obj):
    cache = obj.cache = {}

    @functools.wraps(obj)
    def memoizer(*args, **kwargs):
        key = str(args) + str(kwargs)
        if key not in cache:
            cache[key] = obj(*args, **kwargs)
        return cache[key]
    return memoizer

In [ ]:
class memoize(dict):
    def __init__(self, func):
        self.func = func

    def __call__(self, *args):
        return self[args]

    def __missing__(self, key):
        result = self[key] = self.func(*key)
        return result


In [ ]:
import cProfile

@memoize
def fib(n):
    # from http://en.literateprograms.org/Fibonacci_numbers_(Python)
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)

def fib_seq(n):
    seq = [ ]
    if n > 0:
        seq.extend(fib_seq(n-1))
    seq.append(fib(n))
    return seq

cProfile.run('print(fib_seq(20))')

In [ ]:
import cProfile

@memoize
def fib(n):
    # from http://en.literateprograms.org/Fibonacci_numbers_(Python)
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)

def fib_seq(n):
    seq = [ ]
    if n > 0:
        seq.extend(fib_seq(n-1))
    seq.append(fib(n))
    return seq

cProfile.run('print(fib_seq(20))')

Sometimes, instead of constructing a complex expression for **run()**, it is easier to build a simple expression and pass it parameters through a context, using **runctx()**

In [ ]:
import cProfile

@memoize
def fib(n):
    # from http://en.literateprograms.org/Fibonacci_numbers_(Python)
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)

def fib_seq(n):
    seq = [ ]
    if n > 0:
        seq.extend(fib_seq(n-1))
    seq.append(fib(n))
    return seq

cProfile.runctx('print(fib_seq(n))', globals(), {'n':20})

In this example, the value of **n** is passed through the local variable context instead of being embedded directly in the statement passed to **runctx()**

https://docs.python.org/3/library/functools.html#functools.lru_cache

In [ ]:
from functools import lru_cache

@lru_cache(maxsize=None)
def fib(n):
    if n < 2:
        return n
    return fib(n-1) + fib(n-2)

In [ ]:
[fib(n) for n in range(16)]

In [ ]:
fib.cache_info()

In [ ]:
def cisla1(n=10**1):
    res = '0'
    for i in range(1,n):
        res = res + '; ' + str(i)
    return res

In [ ]:
def cisla2(n=10**1):
    res = '; '.join(str(i) for i in range(n))
    return res

In [ ]:
print(cisla1())
print(cisla2())

In [ ]:
import cProfile

cProfile.runctx('cisla2(10**5)', globals(), locals())
cProfile.runctx('cisla1(10**5)', globals(), locals())

## Memory profiler

In [ ]:
pip install memory_profiler

conda install memory_profiler

In [ ]:
# %load memory_test.py
from memory_profiler import profile
import random

@profile
def memory_test():
    numbers = [random.randint(0, 1000) for x in range(10**5)]
    biggest = max(numbers)
    return biggest

memory_test()


In [ ]:
%run memory_test.py

## Context managers

http://effbot.org/zone/python-with-statement.htm

In [ ]:
def set_things_up():
    global bond_girl
    bond_girl = 'Vesper Lynd'
    print('James meets', bond_girl)
    
def do_something():
    print('James has sex with', bond_girl)
    
def tear_things_down():
    print('The villain kills', bond_girl)
    print('James is sad but free')

In [ ]:
set_things_up()
try:
    do_something()
finally:
    tear_things_down()

If you do this a lot, it would be quite convenient if you could put the âset things upâ and âtear things downâ code in a library function, to make it easy to reuse. 

In [ ]:
def controlled_execution(callback):
    set_things_up()
    try:
        callback()
    finally:
        tear_things_down()

def do_something():
    print('James has sex with', bond_girl)

controlled_execution(do_something)

Thatâs a bit verbose, especially if you need to modify local variables

Another approach is to use a one-shot generator, and use the for-in statement to âwrapâ the code

In [ ]:
def controlled_execution():
    set_things_up()
    try:
        yield bond_girl
    finally:
        tear_things_down()

def do_something_with_thing(bond_girl):
    print('James has sex with', bond_girl)
        
for thing in controlled_execution():
    do_something_with_thing(thing)

 Itâs a bit weird to use a loop construct when you know that you only want to execute something once

In [ ]:
class controlled_execution: 
    def __enter__(self): 
        set_things_up()
        return bond_girl
    def __exit__(self, type, value, traceback): 
        tear_things_down()

def do_something_with_thing(bond_girl):
    print('James has sex with', bond_girl)
        
with controlled_execution() as thing:
    do_something_with_thing(thing)


When the âwithâ statement is executed, Python evaluates the expression, calls the `__enter__` method on the resulting value (which is called a âcontext guardâ), and assigns whatever `__enter__` returns to the variable given by as

Python will then execute the code body, and no matter what happens in that code, call the guard objectâs `__exit__` method

In [ ]:
with open('pr.py', 'r') as f:
    for line in f:
        print(line)


In [ ]:
f = open("pr.py") 
f


In [ ]:
f.__enter__()

In [ ]:
f.read(6)

In [ ]:
with open(input_file_path) as inputfile, open(output_file_path, 'w') as outputfile:
    for line in inputfile:
        outputfile.write(process(line))

In [ ]:
f.__exit__(None, None, None)
f.read(10)

As an extra bonus, the __exit__ method can look at the exception, if any, and suppress it or act on it as necessary

To suppress the exception, just return a true value

For example, the following __exit__ method swallows any TypeError, but lets all other exceptions through:

In [ ]:
def __exit__(self, type, value, traceback):
    return isinstance(value, TypeError)

## contextlib

    @contextlib.contextmanager

This function is a decorator that can be used to define a factory function for with statement context managers, without needing to create a class or separate __enter__() and __exit__() methods

    

A simple example (not recommended as a real way of generating HTML!)

In [ ]:
from contextlib import contextmanager

@contextmanager
def tag(name):
    print("<%s>" % name)
    yield
    print("</%s>" % name)

with tag("h1"):
   print("foo")

The function being decorated must return a generator-iterator when called

This iterator must yield exactly one value, which will be bound to the targets in the with statementâs as clause, if any

At the point where the generator yields, the block nested in the with statement is executed

The generator is then resumed after the block is exited

In [ ]:
import threading, subprocess

doproc = lambda: subprocess.Popen(["python", "-c", "'a=1'"], 
                                  stdout=subprocess.PIPE).communicate()

def dothread():
    def run():
        a = 1
    th = threading.Thread(target=run)
    th.start()
    th.join()

%time junk = [dothread() for _ in range(500)]

In [ ]:
%time junk = [doproc() for _ in range(500)]

In [ ]:
import contextlib
import time

@contextlib.contextmanager
def time_print(task_name):
    t = time.time()
    try:
        yield
    finally:
        print(task_name, "took", time.time() - t, "seconds.")

with time_print("processes"):
    [doproc() for _ in range(500)]

with time_print("threads"):
    [dothread() for _ in range(500)]

# Sorting

https://wiki.python.org/moin/HowTo/Sorting

Python lists have a built-in sort() method that modifies the list in-place and a sorted() built-in function that builds a new sorted list from an iterable. 

In [ ]:
a = [5, 2, 3, 1, 4]
print(sorted(a))
print(a)
a.sort()
print(a)

In [ ]:
sorted({1: 'D', 2: 'B', 3: 'B', 4: 'E', 5: 'A'})

In [ ]:
sorted("This is a test string from Andrew".split(), key=str.lower)

The value of the key parameter should be a function that takes a single argument and returns a key to use for sorting purposes

This technique is fast because the key function is called exactly once for each input record

A common pattern is to sort complex objects using some of the object's indices as a key

In [ ]:
student_tuples = [
        ('john', 'A', 15),
        ('jane', 'B', 12),
        ('dave', 'B', 10),
]
sorted(student_tuples, key=lambda student: student[2])   # sort by age

The same technique works for objects with named attributes

In [ ]:
The same technique works for objects with named attributes

In [ ]:
class Student:
        def __init__(self, name, grade, age):
                self.name = name
                self.grade = grade
                self.age = age
        def __repr__(self):
                return repr((self.name, self.grade, self.age))
        def weighted_grade(self):
                return 'CBA'.index(self.grade) / float(self.age)

student_objects = [
        Student('john', 'A', 15),
        Student('jane', 'B', 12),
        Student('dave', 'B', 10),
]
sorted(student_objects, key=lambda student: student.age)   # sort by age

The key-function patterns shown above are very common, so Python provides convenience functions to make accessor functions easier and faster

The operator module has itemgetter, attrgetter, and a methodcaller function

Using those functions, the above examples become simpler and faster

In [ ]:
from operator import itemgetter, attrgetter, methodcaller

sorted(student_tuples, key=itemgetter(2))

In [ ]:
sorted(student_objects, key=attrgetter('age'))

The operator module functions allow multiple levels of sorting

For example, to sort by grade then by age

In [ ]:
sorted(student_tuples, key=itemgetter(1,2))


In [ ]:
sorted(student_objects, key=attrgetter('grade', 'age'))

The third function from the operator module, methodcaller is used in the following example in which the weighted grade of each student is shown before sorting on it

In [ ]:
[(student.name, student.weighted_grade()) for student in student_objects]

In [ ]:
sorted(student_objects, key=methodcaller('weighted_grade'))

Both list.sort() and sorted() accept a reverse parameter with a boolean value

This is using to flag descending sorts

For example, to get the student data in reverse age order

In [ ]:
sorted(student_tuples, key=itemgetter(2), reverse=True)

In [ ]:
sorted(student_objects, key=attrgetter('age'), reverse=True)

Sorts are guaranteed to be stable

That means that when multiple records have the same key, their original order is preserved

In [ ]:
data = [('red', 2), ('blue', 1), ('red', 1), ('blue', 2)]
sorted(data, key=itemgetter(0))

In [ ]:
sorted(student_objects, key=attrgetter('age'))

This wonderful property lets you build complex sorts in a series of sorting steps

For example, to sort the student data by descending grade and then ascending age, do the age sort first and then sort again using grade 

In [ ]:
s = sorted(student_objects, key=attrgetter('age'))  # sort on secondary key
sorted(s, key=attrgetter('grade'), reverse=True)    # now sort on primary key, descending

The Timsort algorithm used in Python does multiple sorts efficiently because it can take advantage of any ordering already present in a dataset. 

The reverse parameter still maintains sort stability (i.e. records with equal keys retain the original order)

Interestingly, that effect can be simulated without the parameter by using the builtin reversed function twice

    

In [ ]:
data = [('red', 1), ('blue', 1), ('red', 2), ('blue', 2)]
print(sorted(data, reverse=True))
print(list(reversed(sorted(reversed(data)))))


To create a standard sort order for a class, just add the appropriate rich comparison methods

In [ ]:
Student.__eq__ = lambda self, other: self.age == other.age
Student.__ne__ = lambda self, other: self.age != other.age
Student.__lt__ = lambda self, other: self.age < other.age
Student.__le__ = lambda self, other: self.age <= other.age
Student.__gt__ = lambda self, other: self.age > other.age
Student.__ge__ = lambda self, other: self.age >= other.age
sorted(student_objects)


For general purpose comparisons, the recommended approach is to define all six rich comparison operators

The functools.total_ordering class decorator makes this easy to implement

In [ ]:
import functools
import inspect
from pprint import pprint

@functools.total_ordering
class MyObject:

    def __init__(self, val):
        self.val = val

    def __eq__(self, other):
        print('  testing __eq__({}, {})'.format(
            self.val, other.val))
        return self.val == other.val

    def __gt__(self, other):
        print('  testing __gt__({}, {})'.format(
            self.val, other.val))
        return self.val > other.val

In [ ]:
print('Methods:\n')
pprint(inspect.getmembers(MyObject, inspect.isfunction))

a = MyObject(1)
b = MyObject(2)

print('\nComparisons:')
for expr in ['a < b', 'a <= b', 'a == b', 'a >= b', 'a > b']:
    print('\n{:<6}:'.format(expr))
    result = eval(expr)
    print('  result of {}: {}'.format(expr, result))

Key functions need not access data internal to objects being sorted

A key function can also access external resources

For instance, if the student grades are stored in a dictionary, they can be used to sort a separate list of student names

In [ ]:
    students = ['dave', 'john', 'jane']
    newgrades = {'john': 'F', 'jane':'A', 'dave': 'C'}
    sorted(students, key=newgrades.__getitem__)

In [ ]:
from random import random
sorted(students, key=lambda _:random())